<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 4.1.1 Linear Regression

## Prediction of House Price Using Linear Regression

### Data

The Sacramento real estate transactions file is a list of 985 real estate transactions in the Sacramento area reported over a five-day period, as reported by the Sacramento Bee.

In [0]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import pandas as pd

from scipy import stats
from matplotlib import pyplot as plt

sns.set_context("poster")
sns.set(rc={'figure.figsize': (10, 6)})
sns.set_style("whitegrid")

%matplotlib inline

### 1. Explore Data

Perform EDA on this dataset. Use what you have learned so far. Be creative. List all your findings.

In [0]:
# Read CSV
realestate_csv = "/Users/faez/Downloads/git-institute-of-data/module4/data/Sacramento_transactions.csv"
# Read CSV
realestatedata = pd.read_csv(realestate_csv)

In [ ]:
# check the layout of the dataframe
realestatedata.shape

In [ ]:
# Look at the headings and the first five rows of the data in the frame
realestatedata.head()

In [ ]:
# Look at the last five rows in the frame
realestatedata.tail()

In [ ]:
# check how many rows in the dataframe are null
realestatedata.isnull().sum()

In [ ]:
# list the datatype of each columns
realestatedata.info()

In [ ]:
# provide a description of the characteristics of the dataset
realestatedata.describe()

In [ ]:
# look at the beds column and check if any values with zero
realestatedata[realestatedata['beds'] == 0]

In [ ]:
# plot the number of houses in each type of residential category as a bar chart
realestatedata['type'].value_counts().plot(kind='bar')

In [ ]:
# plot the number of houses in each suburb as a horizontol bar chart of size 15cm x 10cm
realestatedata['city'].value_counts().sort_values().plot(kind='barh', figsize=(15,10));

In [ ]:
import folium
import pandas as pd

# generate an empty map
# 37.76, -122.45 is the initital location of SF, USA
mapplot = folium.Map(location=[38.575764, -121.478851], tiles="Mapbox Bright", zoom_start=10)

# place a marker at the location of each house on the map
for i in range(0, len(realestatedata.head(1000))):
    folium.Marker(
        location=[realestatedata.iloc[i]['latitude'], realestatedata.iloc[i]['longitude']]
    ).add_to(mapplot)
    
display(mapplot)

### 2. Predict Price

We are going to predict `price` from available information.

#### 2.1 What is the Target Variable? Why?

In [0]:
# ANSWER
Price is the target variable

#### 2.2 List all possible variables which might be Predictor variable.

In [0]:
# number of bed (beds), size of house(sqft), type of residencea(type), number of bathrooms(type)


#### 2.3 Find correlation between variables

Find correlation between variables. Which is the best predictor? Why? State your reason.

In [0]:
# show scatter plot of beds against target variable price
draw_scatter_plot(realestatedata['beds'], realestatedata['price'])

In [ ]:
# show scatter plot of baths against target variable price
draw_scatter_plot(realestatedata['baths'], realestatedata['price'])

In [ ]:
# show scatter plot of house size against target variable price
draw_scatter_plot(realestatedata['sq__ft'], realestatedata['price'])

In [ ]:
realestatedata.corr()

#as you would expect, house size is the best predictor

#### 2.4 What is the simplest way to predict price

So we have information for 985 houses. What is the easiest way to predict price of a house?

In [0]:
# calculate the mean to get the ballpoint figure for the price of a house
realestatedata['y_hat'] = realestatedata['price'].mean()

#plot the house size against the sale price and plot a line through the plot to show the calculated mean house price
X = realestatedata['sq__ft']
Y = realestatedata['price']
Yhat = realestatedata['y_hat']

def plt_size_vs_price(X, Y, Yhat):
    
    fig = plt.figure(figsize=(15,7))
    # get the axis of that figure
    axis1 = plt.gca()

    # plot a scatter plot on it with our data
    axis1.scatter(X, Y, c='b')
    axis1.plot(X, Yhat, color='r');
    plt.show();
    

In [ ]:
plt_size_vs_price(X, Y, Yhat)

#### 2.5 Find Resudals

Find difference between actual value and predicted value. Use absolute value.

In [0]:
res = abs(realestatedata['price'] - realestatedata['y_hat']).mean()
print(res)

#### 2.6  Predict Using Linear Regression

[Simple Linear Regression](https://en.wikipedia.org/wiki/Simple_linear_regression#Fitting_the_regression_line)

### $$\hat{y} = \beta_0 + \beta_1 x$$

> ### $$ \beta_1 = \frac{\sum_{i=1}^n (y_i - \bar{y} ) (x_i - \bar{x} )}{\sum_{i=1}^n (x_i - \bar{x})^2} $$

and

> ### $$ \beta_0 = \bar{y} - \beta_1\bar{x} $$

Given variables:
- $\bar{y}$ : the sample mean of observed values $Y$
- $\bar{x}$ : the sample mean of observed values $X$

##### 2.7 Define X, Y

- X = Predictor Variable 
- Y = Target Variable

In [0]:
X = realestatedata['sq__ft']
Y = realestatedata['price']

##### 2.7.A Calculate beta_1

 ### $$ \beta_1 = \frac{\sum_{i=1}^n (y_i - \bar{y} ) (x_i - \bar{x} )}{\sum_{i=1}^n (x_i - \bar{x})^2} $$

In [0]:
def calculate_beta1(X, Y):
    Xbar = X.mean()
    Ybar = Y.mean()
    
    numerator = np.sum((Y - Ybar) * (X - Xbar)) 
    denominator = np.sum(np.square(X - Xbar)) 
    
    b1 = numerator/denominator
    
    return b1

In [ ]:
beta1 = calculate_beta1(X, Y)
print(beta1)

##### 2.7.B Calculate beta_0

### $$ \beta_0 = \bar{y} - \beta_1\bar{x} $$

In [0]:
def calculate_beta0(X, Y):
    Xbar = X.mean()
    Ybar = Y.mean()
    b0 = Ybar - calculate_beta_1(X, Y) * Xbar
    
    return b0

In [ ]:
beta0 = calculate_beta0(X, Y)
print(beta0)

##### 2.7.C Find Yhat (Predicted Value Using Beta 0 and Beta 1)

### $$\hat{y} = \beta_0 + \beta_1 x$$

In [0]:
def find_yhat(x, b0, b1):
    return (b0 + b1) * x

In [ ]:
Yhat = find_yhat(X, beta0, beta1)
print(Yhat)

##### 2.7.D Find Residuals

### $$ \text{residual}_i = y_i - \hat{y}_i$$

In [0]:
def find_residual(y, yhat):
    res = y - yh
    return res

##### 2.7.E Find R-Square

The coefficient $R^2$ is defined as $$ R^2 = 1 - \frac{SS_{res}}{SS_{tot}} $$, where $$ SS_{res} $$ is the residual sum of squares $$ SS_{res} = \sum_{i=1}^n \left(y_i - \hat{y}_i\right)^2 $$ and $$ SS_{res} $$ is the total sum of squares $$ SS_{tot} = \sum_{i=1}^n \left(y_i - \bar{y}\right)^2 $$

The best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse).

**Write a function to calculate R- Square value.**

The total sum of squares:

### $$ SS_{tot} = \sum_{i=1}^n \left(y_i - \bar{y}\right)^2 $$

The residual sum of squares:

### $$ SS_{res} = \sum_{i=1}^n \left(y_i - \hat{y}_i\right)^2 $$

$R^2$:

### $$ R^2 = 1 - \frac{SS_{res}}{SS_{tot}} $$

In [0]:
def total_sum_of_square(Y):
    Ybar = np.mean(Y)
    SStot = np.sum((Y-Ybar)**2)
    return SStot

In [ ]:
def residual_sum_of_square(Y, Yhat):
    SSres = np.sum((Y - Yhat)**2)
    return SSres

In [ ]:
def r_square(Y, Yhat):
    SStot = total_sum_of_square(Y)
    SSres = residual_sum_of_square(Y, Yhat)
    Rs2 = 1 - (SSres/SStot)
    return Rs2

In [ ]:
Rsquared = r_square(Y,Yhat)

##### 2.7.F [BONUS] Draw Plot of Predicted Line

In [0]:
# ANSWER

##### 2.7.G [BONUS] Draw Plot of Predicted Line With Resudals

In [0]:
# ANSWER

### 3. Predict House Price

Let's assume we have information of following house:

- street:	1140 EDMONTON DR
- city:	SACRAMENTO
- zip:	95833
- state:	CA
- beds:	3
- baths:	2
- sq__ft:	1204
- type:	Residential

**Predict the price of this house.**

In [0]:
# ANSWER



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



